# Create climate dataset for capitals in Europe

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

# to resolve xarray FutureWarning
xr.set_options(use_new_combine_kwarg_defaults=True)

In [2]:
# mapping between variable names in ISIMIP data and ERA5 dataset

mappings = {
    'tas': 't2m',
    'pr': 'tp',
    'prsn': 'sf',       
    'sfcwind': 'si10',
    'tasmax': 'max_t2m',
    'tasmin': 'min_t2m',
}

In [3]:
# Check all available datasets first
eu_capitals = pd.read_csv("datasets/european_cities.csv")
print(eu_capitals.shape)
eu_capitals.head()

(300, 3)


,name,latitude,longitude
0,Aachen,50.776642,6.08342
1,Aberdeen,57.143688,-2.09814
2,Aix-en-Provence,43.528301,5.44973
3,Alcalá de Henares,40.482052,-3.35996
4,Alicante,38.345169,-0.48149


In [4]:
bern = eu_capitals[eu_capitals['name'] == 'Bern']
display(bern)

,name,latitude,longitude
26,Bern,46.94809,7.44744


In [5]:
periods = ['2021_2030', '2031_2040', '2041_2050']
features = ['pr', 'prsn', 'sfcwind', 'tas', 'tasmax', 'tasmin']
datasets = {}

for feature in features:
    temp_datasets = []
    
    for period in periods:
        file_path = f"isimip3b_IPSL_CM6A_LR/ssp370/{feature}/ipsl-cm6a-lr_r1i1p1f1_w5e5_ssp370_{feature}_global_daily_{period}.nc"
        ds = xr.open_dataset(file_path)
        temp_datasets.append(ds)
    
    # Concatenate along the time dimension for the full 2021-2050 period
    datasets[f"{feature}_2021_2050"] = xr.concat(temp_datasets, dim='time')
    print(f"Created {feature}_2021_2050 with {datasets[f'{feature}_2021_2050'].sizes['time']} time steps")

print("\nAvailable datasets:")
print(datasets.keys())
print(f"\nExample - pr_2021_2050 time range:")
print(f"Start: {datasets['pr_2021_2050']['time'].values[0]}")
print(f"End: {datasets['pr_2021_2050']['time'].values[-1]}")

Created pr_2021_2050 with 10957 time steps
Created prsn_2021_2050 with 10957 time steps
Created sfcwind_2021_2050 with 10957 time steps
Created tas_2021_2050 with 10957 time steps
Created tasmax_2021_2050 with 10957 time steps
Created tasmin_2021_2050 with 10957 time steps

Available datasets:
dict_keys(['pr_2021_2050', 'prsn_2021_2050', 'sfcwind_2021_2050', 'tas_2021_2050', 'tasmax_2021_2050', 'tasmin_2021_2050'])

Example - pr_2021_2050 time range:
Start: 2021-01-01T12:00:00.000000000
End: 2050-12-31T12:00:00.000000000


In [6]:
periods = ['2021_2050']

print("Nb of latitudes:", datasets['tas_2021_2050']['lat'].size)
print("Nb of longitudes:", datasets['tas_2021_2050']['lon'].size)
print("Time period:", str(datasets['tas_2021_2050']['time'].values[0])[:10], "to", str(datasets['tas_2021_2050']['time'].values[-1])[:10])

Nb of latitudes: 360
Nb of longitudes: 720
Time period: 2021-01-01 to 2050-12-31


## Keep only data in an area around Europe (lat: 35 to 72, lon: -25 to 45)

In [7]:
def keep_area_europe(dataset):
    """Keep only data in an area around Europe (lat: 35 to 72, lon: -25 to 45)"""
    europe_data = dataset.sel(lat=slice(72, 35), lon=slice(-25, 45))
    return europe_data

In [8]:
ds_europe = {}

for feature in features:
    for period in periods:
        ds_europe[f"{feature}_{period}"] = keep_area_europe(datasets[f"{feature}_{period}"])

In [9]:
ds_europe['tas_2021_2050']

<xarray.Dataset> Size: 454MB
Dimensions:  (time: 10957, lat: 74, lon: 140)
Coordinates:
  * time     (time) datetime64[ns] 88kB 2021-01-01T12:00:00 ... 2050-12-31T12...
  * lat      (lat) float64 592B 71.75 71.25 70.75 70.25 ... 36.25 35.75 35.25
  * lon      (lon) float64 1kB -24.75 -24.25 -23.75 -23.25 ... 43.75 44.25 44.75
Data variables:
    tas      (time, lat, lon) float32 454MB 257.1 255.9 256.2 ... 286.7 284.7
Attributes:
    title:        ISIMIP3b bias-adjusted climate input data
    institution:  Potsdam Institute for Climate Impact Research (PIK)
    project:      Inter-Sectoral Impact Model Intercomparison Project phase 3...
    contact:      ISIMIP cross-sectoral science team <info@isimip.org> <https...
    summary:      CMIP6 daily output data bias-adjusted and statistically dow...
    references:   Lange (2019) <https://doi.org/10.5194/gmd-12-3055-2019> and...

## Convert units to match ERA5 dataset (precipitation and snowfall from kg m-2 s-1 to m)

#### Units comparison between ERA5 and ISIMIP3b:

| Feature             | ERA5 Unit  | ISIMIP3b Unit |
| ------------------- | ---------- | ------------- |
| 2m Temperature      | Kelvin (K) | K             |
| wind speed          | m s^-1    | m s^-1        |
| total precipitation | m          | kg m^-2 s^-1  |
| snowfall            | m          | kg m^-2 s^-1  |

- 1 kg m⁻² = 1 mm water
- precipitation rate is for one day -> multiply by 86400 (seconds in a day) and convert mm to m (divide by 1000)

In [10]:
def convert_precipitation_units(dataset):
    """Convert precipitation and snowfall from kg m-2 s-1 to m"""
    return dataset / 1000 * 86400  # Convert from kg m-2 s-1 to m

In [11]:
features_to_convert = ['pr', 'prsn']

for feature in features_to_convert:
    for period in periods:
        print(f"Converting units for {feature} in period {period}")
        ds_europe[f"{feature}_{period}"] = convert_precipitation_units(ds_europe[f"{feature}_{period}"])

Converting units for pr in period 2021_2050
Converting units for prsn in period 2021_2050


## Keep data within a radius of 50km (0.45 degrees) around each capital

In [12]:
def reduce_dataset_to_cities(dataset, cities_df, radius_deg=0.45) -> xr.Dataset:
    """Fully vectorized version using broadcasting."""
    
    # Extraire les coordonnées
    lats_cities = cities_df['latitude'].values
    lons_cities = cities_df['longitude'].values
    names = cities_df['name'].values
    
    lat_grid = dataset['lat'].values
    lon_grid = dataset['lon'].values
    
    # Charger les données en mémoire (si possible)
    data_vars = list(dataset.data_vars)
    
    # Préparer les listes pour stocker les villes valides
    n_time = dataset.sizes['time']
    valid_indices = []
    valid_names = []
    valid_lats = []
    valid_lons = []
    results_dict = {var: [] for var in data_vars}
    
    # Treat each city in one loop
    for i, (lat_city, lon_city, name) in enumerate(zip(lats_cities, lons_cities, names)):
        # booleans masks for lat/lon within radius
        lat_mask = (lat_grid >= lat_city - radius_deg) & (lat_grid <= lat_city + radius_deg)
        lon_mask = (lon_grid >= lon_city - radius_deg) & (lon_grid <= lon_city + radius_deg)
        
        n_points = np.sum(lat_mask) * np.sum(lon_mask)

        if n_points == 0:
            print(f"Warning: No grid points found within radius {radius_deg} degrees for city {name}. Skipping.")
            continue
        
        # Ville valide - ajouter aux listes
        valid_indices.append(i)
        valid_names.append(name)
        valid_lats.append(lat_city)
        valid_lons.append(lon_city)
        
        # Extraire et moyenner pour chaque variable
        for var in data_vars:
            data_subset = dataset[var].values[:, lat_mask][:, :, lon_mask]
            averaged_data = np.nanmean(data_subset, axis=(1, 2))
            results_dict[var].append(averaged_data)
    
    # Convertir les listes en arrays numpy
    valid_names = np.array(valid_names)
    valid_lats = np.array(valid_lats)
    valid_lons = np.array(valid_lons)
    
    for var in data_vars:
        results_dict[var] = np.column_stack(results_dict[var])  # (n_time, n_valid_cities)
    
    # Créer le dataset de sortie avec time et city comme dimensions principales
    data_arrays = {}
    for var in data_vars:
        data_arrays[var] = xr.DataArray(
            results_dict[var],
            dims=('time', 'city'),
            coords={
                'time': dataset['time'],
                'city': valid_names
            }
        )
    
    result_ds = xr.Dataset(data_arrays)
    
    # Ajouter latitude et longitude comme coordonnées indexées sur la dimension 'city'
    result_ds = result_ds.assign_coords({
        'latitude': ('city', valid_lats),
        'longitude': ('city', valid_lons)
    })
    
    print(f"Processed {len(valid_names)} cities out of {len(names)} total.")
    
    return result_ds

In [13]:
# Faire en sorte de passer une seule fois par époque donc toutes les variables d'un coup

ds_reduced = {}

for feature in features:
    for period in periods:
        print(f"Processing {feature} for period {period}...")
        ds_reduced[f"{feature}_{period}"] = reduce_dataset_to_cities(
            ds_europe[f"{feature}_{period}"],
            eu_capitals,
            radius_deg=0.45
        )

Processing pr for period 2021_2050...
Processed 292 cities out of 300 total.
Processing prsn for period 2021_2050...
Processed 292 cities out of 300 total.
Processing sfcwind for period 2021_2050...
Processed 292 cities out of 300 total.
Processing tas for period 2021_2050...
Processed 292 cities out of 300 total.
Processing tasmax for period 2021_2050...
Processed 292 cities out of 300 total.
Processing tasmin for period 2021_2050...
Processed 292 cities out of 300 total.


In [14]:
ds_reduced['tas_2021_2050']

<xarray.Dataset> Size: 13MB
Dimensions:    (time: 10957, city: 292)
Coordinates:
  * time       (time) datetime64[ns] 88kB 2021-01-01T12:00:00 ... 2050-12-31T...
  * city       (city) <U24 28kB 'Aachen' 'Aberdeen' ... 'Århus' 'Łódź'
    latitude   (city) float64 2kB 50.78 57.14 43.53 40.48 ... 47.37 56.16 51.75
    longitude  (city) float64 2kB 6.083 -2.098 5.45 -3.36 ... 8.55 10.21 19.47
Data variables:
    tas        (time, city) float32 13MB 275.8 276.5 280.1 ... 274.3 275.7 274.7

In [15]:
# Verify dimension
print("Dataset dimensions before reduction:", datasets['tas_2021_2050'].dims)
print("Dataset dimensions after reduction:", ds_reduced['tas_2021_2050'].dims)

Dataset dimensions before reduction: FrozenMappingWarningOnValuesAccess({'time': 10957, 'lat': 360, 'lon': 720})
Dataset dimensions after reduction: FrozenMappingWarningOnValuesAccess({'time': 10957, 'city': 292})


## Find nb days above threshold per year

In [16]:
def find_nb_days_above_threshold_per_year(da, threshold):
    """Find number of days above a temperature threshold per year."""
    # Convert threshold from Celsius to Kelvin
    threshold_k = threshold + 273.15
    
    # Create a boolean DataArray where True indicates temperature above threshold
    above_threshold = da > threshold_k
    
    # Group by year and sum the number of days above the threshold
    nb_days_per_year = above_threshold.groupby('time.year').sum(dim='time')
    
    return nb_days_per_year

def compute_avg_hot_days_per_city(ds_reduced, period, threshold=30):
    """
    Compute average number of days above threshold per year for each city over the entire period.
    
    Args:
        ds_reduced: dict with reduced datasets
        period: string like '2021_2050'
        threshold: temperature threshold in Celsius
    
    Returns:
        dict: {city_name: avg_days_above_threshold}
    """
    # Get tasmax data
    tasmax = ds_reduced[f'tasmax_{period}']['tasmax']
    
    # Find days above threshold per year for each city
    days_per_year = find_nb_days_above_threshold_per_year(tasmax, threshold)
    
    # Calculate mean across all years for each city
    avg_days_per_city = days_per_year.mean(dim='year')
    
    # Convert to dictionary
    result = {str(city): float(avg_days_per_city.sel(city=city).values) 
              for city in avg_days_per_city.city.values}
    
    return result

In [17]:
avg_days_above_30C_per_city = compute_avg_hot_days_per_city(ds_reduced, '2021_2050', threshold=30)
avg_days_above_30C_per_city['Bern']

5.866666666666666

## Find nb consecutive days above threshold per year

In [18]:
def max_consecutive_days(arr):
    """Helper function to find the maximum number of consecutive True values in a 1D boolean array."""
    max_count = 0
    current_count = 0
    
    for value in arr:
        if value:
            current_count += 1
            max_count = max(max_count, current_count)
        else:
            current_count = 0
            
    return max_count

def find_max_consecutive_days_per_year(da, threshold):
    """Find maximum number of consecutive days above a temperature threshold per year for each city.

    Args:
        da: xarray DataArray with dimensions (time, city)
        threshold: temperature threshold in Celsius
    Returns:
        xarray DataArray with dimensions (year, city) containing max consecutive days per year
    """
    threshold_k = threshold + 273.15
    above_threshold = da > threshold_k
    
    # Group by year
    grouped = above_threshold.groupby('time.year')
    
    # For each year and each city, find max consecutive days
    results = []
    years = []
    
    for year, group in grouped:
        years.append(year)
        # group has dimensions (time, city)
        year_results = []
        
        for city_idx in range(group.sizes['city']):
            # Extract 1D array for this city
            city_data = group.isel(city=city_idx).values
            max_consec = max_consecutive_days(city_data)
            year_results.append(max_consec)
        
        results.append(year_results)
    
    # Create DataArray with proper dimensions
    result_da = xr.DataArray(
        results,
        dims=['year', 'city'],
        coords={
            'year': years,
            'city': da.city.values
        }
    )
    
    return result_da

def compute_avg_consec_hot_days_per_city(ds_reduced, period, threshold=30):
    """
    Compute average maximum consecutive days above threshold per year for each city.
    
    Args:
        ds_reduced: dict with reduced datasets
        period: string like '2021_2050'
        threshold: temperature threshold in Celsius
    Returns:
        dict: {city_name: avg_max_consecutive_days_above_threshold}
    """
    # Get tasmax data
    tasmax = ds_reduced[f'tasmax_{period}']['tasmax']
    
    # Find max consecutive days per year for each city
    consec_days_per_year = find_max_consecutive_days_per_year(tasmax, threshold)
    
    # Calculate mean across all years for each city
    avg_consec_days_per_city = consec_days_per_year.mean(dim='year')
    
    # Convert to dictionary with proper string keys
    result = {}
    for city in avg_consec_days_per_city.city.values:
        city_name = city.item() if hasattr(city, 'item') else str(city)
        result[city_name] = float(avg_consec_days_per_city.sel(city=city).values)
    
    return result

In [19]:
avg_consecutive_days_above_30C_per_city = compute_avg_consec_hot_days_per_city(ds_reduced, '2021_2050', threshold=30)
avg_consecutive_days_above_30C_per_city['Bern']

3.3666666666666667

## Aggregate daily data to seasonal data

In [20]:
def aggregate_per_season(dataset):
    """Aggregate daily data to seasonal data by taking the mean over each season."""
    seasonal_data = dataset.resample(time='QS-DEC').mean()
    return seasonal_data

In [21]:
ds_seasonal = {}

for feature in features:
    for period in periods:
        ds_seasonal[f"{feature}_{period}"] = aggregate_per_season(ds_reduced[f"{feature}_{period}"])

In [22]:
ds_seasonal['tas_2021_2050']

<xarray.Dataset> Size: 175kB
Dimensions:    (time: 121, city: 292)
Coordinates:
  * time       (time) datetime64[ns] 968B 2020-12-01 2021-03-01 ... 2050-12-01
  * city       (city) <U24 28kB 'Aachen' 'Aberdeen' ... 'Århus' 'Łódź'
    latitude   (city) float64 2kB 50.78 57.14 43.53 40.48 ... 47.37 56.16 51.75
    longitude  (city) float64 2kB 6.083 -2.098 5.45 -3.36 ... 8.55 10.21 19.47
Data variables:
    tas        (time, city) float32 141kB 274.7 276.3 278.5 ... 277.3 276.8

In [23]:
# Verification
print(f"Original dataset time points: {ds_reduced['tas_2021_2050'].sizes['time']}")
# print(f"Monthly aggregated dataset time points: {ds_monthly['tas_2021_2050'].sizes['time']}")
print(f"Seasonal aggregated dataset time points: {ds_seasonal['tas_2021_2050'].sizes['time']}")

Original dataset time points: 10957
Seasonal aggregated dataset time points: 121


# Average climate features for each decade

For each feature, for each capital (lat,lon), do the mean on 10 years for each month. For example, for the feature "2m temperature" (t2m), for the capital of France (48.8566, 2.3522), do the mean of all January data from 1970 to 1979, then do the mean of all February data from 1970 to 1979, and so on until December. Then repeat the same operation for the periods 1980-1989, 1990-1999, 2000-2009 and 2010-2020. In the end, we will have for each year, a dataset with 2 + 12 * 4 = 50 columns: latitude, longitude, and for each month (12) the 4 features (sf, tp, t2m, si10).

<img src="images/ds_columns_past_v2.png" style="width:1000px;"  />


In [24]:
def average_over_period(dataset, month) -> xr.Dataset:
    """Average the dataset over a specified time period."""
    period_data = dataset.sel(time=dataset['time.month'] == month)
    averaged_data = period_data.mean(dim='time')
    return averaged_data

In [25]:
ds_periods = {}
months = [12, 3, 6, 9]  # winter, spring, summer, autumn

for period in periods:
    for feature in features:
        ds_season = ds_seasonal[f"{feature}_{period}"]
        ds_result = xr.Dataset()
        for month in months:
            ds_month = average_over_period(
                ds_season,
                month
            )
            if ds_result.sizes:
                ds_result = xr.concat([ds_result, ds_month], dim='time')
            else:
                ds_result = ds_month
        ds_periods[f"{feature}_{period}"] = ds_result

In [26]:
ds_periods.keys()

dict_keys(['pr_2021_2050', 'prsn_2021_2050', 'sfcwind_2021_2050', 'tas_2021_2050', 'tasmax_2021_2050', 'tasmin_2021_2050'])

In [27]:
# print average seasonal tas for Bern in period 2021-2030

ds_periods['tas_2021_2050']['tas'].sel(city='Bern')

<xarray.DataArray 'tas' (time: 4)> Size: 16B
array([275.16138, 281.8691 , 291.13535, 283.7002 ], dtype=float32)
Coordinates:
    city       <U24 96B 'Bern'
    latitude   float64 8B 46.95
    longitude  float64 8B 7.447
Dimensions without coordinates: time

In [28]:
ds_periods.keys()

dict_keys(['pr_2021_2050', 'prsn_2021_2050', 'sfcwind_2021_2050', 'tas_2021_2050', 'tasmax_2021_2050', 'tasmin_2021_2050'])

## Reconstruct the dataframe with all features for all decades

In [29]:
def aggregate_feature_per_period(features, period, datasets, hot_days, hot_days_consec) -> pd.DataFrame:
    """Aggregate features per specified seasons and return a DataFrame."""
    records = []
    
    # Récupérer les villes et coordonnées depuis le premier dataset
    first_ds = datasets[f"{features[0]}_{period}"]
    cities = first_ds['city'].values
    lats = first_ds['latitude'].values
    lons = first_ds['longitude'].values
    
    # Noms des saisons correspondant aux mois [12, 3, 6, 9]
    season_names = ['winter', 'spring', 'summer', 'autumn']
    
    # Créer une ligne par ville
    for city_idx, city in enumerate(cities):
        record = {
            'city': city,
            'latitude': lats[city_idx],
            'longitude': lons[city_idx],
            'hot_days': hot_days[city],
            'hot_days_consec': hot_days_consec[city],
        }
        
        tasmax_data = None
        tasmin_data = None
        
        # Ajouter toutes les features pour toutes les saisons
        for feature in features:
            ds_key = f"{feature}_{period}"
            ds = datasets[ds_key]
            
            feature_data = ds[feature].sel(city=city).values
            
            # Stocker tasmax et tasmin pour calculer temp_range après
            if feature == 'tasmax':
                tasmax_data = feature_data
            elif feature == 'tasmin':
                tasmin_data = feature_data
            
            # Si on a 4 valeurs (une par saison), les assigner aux colonnes appropriées
            if len(feature_data) == 4:
                for season_idx, season_name in enumerate(season_names):
                    record[f'{mappings[feature]}_{season_name}'] = feature_data[season_idx]
            else:
                # Fallback: si la structure est différente, prendre la moyenne
                record[f'{mappings[feature]}'] = feature_data.mean()
        
        # Add temp_range columns (tmax-tmin) for each season
        if tasmax_data is not None and tasmin_data is not None:
            for season_idx, season_name in enumerate(season_names):
                temp_range = tasmax_data[season_idx] - tasmin_data[season_idx]
                record[f'temp_range_{season_name}'] = temp_range

        records.append(record)
    
    df = pd.DataFrame(records)
    return df

In [30]:
final_dfs = {}

for period in periods:
    df_period = aggregate_feature_per_period(features, period, ds_periods, avg_days_above_30C_per_city, avg_consecutive_days_above_30C_per_city)
    final_dfs[period] = df_period

In [31]:
final_dfs['2021_2050'].head()

,city,latitude,longitude,hot_days,hot_days_consec,tp_winter,tp_spring,tp_summer,tp_autumn,sf_winter,...,max_t2m_summer,max_t2m_autumn,min_t2m_winter,min_t2m_spring,min_t2m_summer,min_t2m_autumn,temp_range_winter,temp_range_spring,temp_range_summer,temp_range_autumn
0,Aachen,50.776642,6.08342,10.633333,4.000000,0.003185,0.002094,0.002427,0.002466,0.000320,...,297.144196,289.563751,274.908630,278.708160,286.555389,281.503052,5.650726,9.335938,10.588806,8.060699
1,Aberdeen,57.143688,-2.09814,0.000000,0.000000,0.002553,0.001769,0.002016,0.002913,0.000153,...,290.280365,286.060059,276.178955,278.079407,284.107635,280.988861,4.437469,5.751434,6.172729,5.071198
2,Aix-en-Provence,43.528301,5.44973,31.100000,13.466667,0.001714,0.001984,0.000845,0.003320,0.000019,...,301.399048,293.917297,277.913147,282.103821,291.014496,286.112762,6.869324,8.843964,10.384552,7.804535
3,Alcalá de Henares,40.482052,-3.35996,81.233333,44.766667,0.001411,0.001541,0.000577,0.001682,0.000156,...,305.870880,295.204895,275.666901,280.036499,289.770233,283.396149,10.200989,12.315613,16.100647,11.808746
4,Alicante,38.345169,-0.48149,88.366667,59.700000,0.001155,0.001302,0.000353,0.001654,0.000015,...,305.543884,298.564270,279.605316,283.395599,293.030853,287.666473,10.817810,11.404785,12.513031,10.897797


In [32]:
# Check for any missing values in each period dataset
for period_name, df in final_dfs.items():
    missing_counts = df.isnull().sum()
    total_missing = missing_counts.sum()
    print(f"\n{period_name}:")
    print(f"  Total missing values: {total_missing}")
    if total_missing > 0:
        print("  Missing values per column:")
        print(missing_counts[missing_counts > 0])
        print("Cities with missing values:")
        print(df[df.isnull().any(axis=1)]['city'].values)


2021_2050:
  Total missing values: 0


In [33]:
# Sort by city name for consistency

for period in periods:
    final_dfs[period] = final_dfs[period].sort_values(by='city').reset_index(drop=True)

## Save the datasets as CSV files

In [34]:
mapppings = {
    '2021_2050': '2021_2050',
}

for period_name, df in final_dfs.items():
    df.to_csv(f"datasets/climate_features_{mapppings[period_name]}_future_ssp370.csv", index=False)
    print(f"Saved climate_features_{mapppings[period_name]}_future_ssp370.csv")

Saved climate_features_2021_2050_future_ssp370.csv
